# Tools for Analytics Final Project ###

Jaeseop Shin / js6364  &. Hyunjin Jun / hj2642

## Environment Setup

In [23]:
# Importing all libraries used for the project

import numpy as np
import math
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import geopandas as gpd
import requests
import psycopg2
import pathlib
import os
import shapely

from bs4 import BeautifulSoup
from sodapy import Socrata
from sqlalchemy import create_engine, text
from shapely.geometry import Point, Polygon
from geoalchemy2 import Geometry, WKTElement
from shapely.geometry.base import BaseGeometry

# SQL Data
db_username = "ariez"
db_name = "postgres"
db_url = f"postgresql+psycopg2://{db_username}@localhost/{db_name}"
db_schema = "schema.sql"
# directory where DB queries for Part 3 will be saved
QUERY_DIR = pathlib.Path("queries")

# Make sure the QUERY_DIRECTORY exists
if not QUERY_DIR.exists():
    QUERY_DIR.mkdir()

## Part 1. Data Processing

### Data Setup

In [2]:
app_token = "2Hn2wwabCLXVYhGN4b9tEtJ11"


# Downloading NYC 311 data
url_311 = "data.cityofnewyork.us"
set_311 = "erm2-nwe9"
client_311 = Socrata(url_311, app_token)
client_311.timeout = 60
# Filtering data after 2015
where_311 = "date_extract_y(created_date)>=2015"
# Data to data frame
data_311 = client_311.get(set_311, where=where_311, limit=20)
df_311 = pd.DataFrame.from_records(data_311)
#df_311.to_csv("nyc_311.csv")

# Downloading NYC Tree data
url_tree = "data.cityofnewyork.us"
set_tree = "uvpi-gqnh"
client_tree = Socrata(url_tree, app_token)
client_tree.timeout = 60
# Data to data frame
data_tree = client_tree.get(set_tree, limit=20)
df_tree = pd.DataFrame.from_records(data_tree)
#df_tree.to_csv("nyc_tree.csv")

# Loading local data files
data_dir = pathlib.Path("data")
rent_path = data_dir / "zillow_rent_data.csv"          
df_rent = pd.read_csv(rent_path)
nyc_zipcodes_shp_path = data_dir / "nyc_zipcodes.shp"
gdf_zipcode = gpd.read_file(nyc_zipcodes_shp_path)
gdf_zipcode = gdf_zipcode.to_crs("EPSG:4326")

In [3]:
df_311

,unique_key,created_date,agency,agency_name,complaint_type,descriptor,location_type,incident_zip,incident_address,street_name,...,open_data_channel_type,park_facility_name,park_borough,latitude,longitude,location,intersection_street_1,intersection_street_2,facility_type,closed_date
0,59551848,2023-11-27T12:00:00.000,DSNY,Department of Sanitation,Derelict Vehicles,Derelict Vehicles,Street,11370,31-42 81 STREET,81 STREET,...,PHONE,Unspecified,QUEENS,40.75843702556503,-73.88684933571109,"{'latitude': '40.75843702556503', 'longitude':...",NaN,NaN,NaN,NaN
1,59551851,2023-11-27T12:00:00.000,DSNY,Department of Sanitation,Derelict Vehicles,Derelict Vehicles,Street,10468,50 EAST 191 STREET,EAST 191 STREET,...,PHONE,Unspecified,BRONX,40.863974007488046,-73.89820995792564,"{'latitude': '40.863974007488046', 'longitude'...",NaN,NaN,NaN,NaN
2,59551328,2023-11-27T00:56:00.000,DOT,Department of Transportation,Traffic Signal Condition,Controller,NaN,11217,NaN,NaN,...,UNKNOWN,Unspecified,BROOKLYN,40.687024872839224,-73.97620565382692,"{'latitude': '40.687024872839224', 'longitude'...",LAFAYETTE AVENUE,FULTON STREET,N/A,NaN
3,59554861,2023-11-27T00:41:00.000,DSNY,Department of Sanitation,Adopt-A-Basket,Adopt-A-Basket,Sidewalk,11429,210-22 HOLLIS AVENUE,HOLLIS AVENUE,...,PHONE,Unspecified,QUEENS,40.708826311311675,-73.7473847324225,"{'latitude': '40.708826311311675', 'longitude'...",NaN,NaN,NaN,NaN
4,59553569,2023-11-27T00:34:00.000,DOT,Department of Transportation,Traffic Signal Condition,Controller,NaN,10474,NaN,NaN,...,UNKNOWN,Unspecified,BRONX,40.81941934884539,-73.88391311273236,"{'latitude': '40.81941934884539', 'longitude':...",EDGEWATER ROAD,SENECA AVENUE,N/A,NaN
5,59552852,2023-11-27T00:30:00.000,DOT,Department of Transportation,Traffic Signal Condition,Controller,NaN,NaN,NaN,NaN,...,UNKNOWN,Unspecified,MANHATTAN,NaN,NaN,NaN,7 AVE,155 ST W,N/A,NaN
6,59552850,2023-11-27T00:21:00.000,DOT,Department of Transportation,Traffic Signal Condition,Controller,NaN,11219,NaN,NaN,...,UNKNOWN,Unspecified,BROOKLYN,40.635848063870505,-73.99483331950307,"{'latitude': '40.635848063870505', 'longitude'...",NEW UTRECHT AVENUE,12 AVENUE,N/A,NaN
7,59555105,2023-11-27T00:06:00.000,DOT,Department of Transportation,Street Light Condition,Street Light Lamp Missing,NaN,11213,1493 PRESIDENT STREET,PRESIDENT STREET,...,UNKNOWN,Unspecified,BROOKLYN,40.66743534786828,-73.9388428031135,"{'latitude': '40.66743534786828', 'longitude':...",NaN,NaN,NaN,NaN
8,59550961,2023-11-26T23:52:36.000,DOT,Department of Transportation,Street Condition,Pothole,NaN,11365,182 STREET,182 STREET,...,UNKNOWN,Unspecified,QUEENS,NaN,NaN,NaN,NaN,NaN,N/A,NaN
9,59551327,2023-11-26T23:50:00.000,DOT,Department of Transportation,Traffic Signal Condition,Controller,NaN,10306,NaN,NaN,...,UNKNOWN,Unspecified,STATEN ISLAND,40.568763404060604,-74.11148321326283,"{'latitude': '40.568763404060604', 'longitude'...",HYLAN BOULEVARD,ROSS AVENUE,N/A,NaN


### Data Cleaning & Filtering 

In [9]:
def clean_filter(data_frame, column_use, column_rename, column_type):
    """Removes unnecessary columns, rename columns, confirm types of columns, discard invalid data"""
    data_frame = data_frame[column_use]
    data_frame = data_frame.rename(columns=column_rename)
    for column, dtype in column_type.items():
        if dtype == 'datetime':
            data_frame[column] = pd.to_datetime(data_frame[column], errors='coerce')
        else:
            data_frame[column] = data_frame[column].astype(dtype, errors='ignore')
    data_frame.dropna(inplace=True)
    return data_frame

In [10]:
# NYC 311 Cleaning & Filtering
# Columns to use
use_311 = [
    'unique_key',
    'created_date',
    'complaint_type',
    'incident_zip',
    'city',
    'x_coordinate_state_plane',
    'y_coordinate_state_plane',
    'latitude',
    'longitude',
]
# Columns to rename
rename_311 = {
    'unique_key': 'complaint_id',
    'created_date': 'date',
    'complaint_type': 'complaint_type',
    'incident_zip': 'zipcode',
    'city': 'city',
    'x_coordinate_state_plane': 'x_coord',
    'y_coordinate_state_plane': 'y_coord',
    'latitude': 'latitude',
    'longitude': 'longitude',
}
# Column types
type_311 = {
    'complaint_id': 'int',
    'date': 'datetime',
    'complaint_type': 'str',
    'zipcode': 'int',
    'city': 'str',
    'x_coord': 'float',
    'y_coord': 'float',
    'latitude': 'float',
    'longitude': 'float',
}

df_311_f = clean_filter(df_311, use_311, rename_311, type_311)

# Converting latitude and longitude into Geometry data
df_311_f['geometry'] = df_311_f.apply(lambda x: Point(float(x['longitude']), float(x['latitude'])), axis=1)

# Converting DataFrame to GeoDataFrame
gdf_311_f = gpd.GeoDataFrame(df_311_f, geometry='geometry')

# Setting the coordinate reference system
gdf_311_f.set_crs(epsg=4326, inplace=True)
gdf_311_f

,complaint_id,date,complaint_type,zipcode,city,x_coord,y_coord,latitude,longitude,geometry
0,59551848,2023-11-27 12:00:00,Derelict Vehicles,11370,EAST ELMHURST,1015597.0,215616.0,40.758437,-73.886849,POINT (-73.88685 40.75844)
1,59551851,2023-11-27 12:00:00,Derelict Vehicles,10468,BRONX,1012405.0,254063.0,40.863974,-73.898210,POINT (-73.89821 40.86397)
2,59551328,2023-11-27 00:56:00,Traffic Signal Condition,11217,BROOKLYN,990849.0,189579.0,40.687025,-73.976206,POINT (-73.97621 40.68702)
3,59554861,2023-11-27 00:41:00,Adopt-A-Basket,11429,QUEENS VILLAGE,1054286.0,197622.0,40.708826,-73.747385,POINT (-73.74738 40.70883)
4,59553569,2023-11-27 00:34:00,Traffic Signal Condition,10474,BRONX,1016381.0,237835.0,40.819419,-73.883913,POINT (-73.88391 40.81942)
6,59552850,2023-11-27 00:21:00,Traffic Signal Condition,11219,BROOKLYN,985684.0,170933.0,40.635848,-73.994833,POINT (-73.99483 40.63585)
7,59555105,2023-11-27 00:06:00,Street Light Condition,11213,BROOKLYN,1001216.0,182447.0,40.667435,-73.938843,POINT (-73.93884 40.66744)
9,59551327,2023-11-26 23:50:00,Traffic Signal Condition,10306,STATEN ISLAND,953277.0,146512.0,40.568763,-74.111483,POINT (-74.11148 40.56876)
10,59555466,2023-11-26 23:48:38,Street Condition,11365,QUEENS,1041671.0,209376.0,40.741179,-73.792786,POINT (-73.79279 40.74118)
11,59552849,2023-11-26 23:43:00,Traffic Signal Condition,11433,QUEENS,1040143.0,195368.0,40.702740,-73.798416,POINT (-73.79842 40.70274)


In [11]:
# NYC Tree Cleaning & Filtering
# Columns to use
use_tree = [
    'tree_id',
    'status',
    'health',
    'spc_common',
    'zipcode',
    'zip_city',
    'latitude',
    'longitude',
    'x_sp',
    'y_sp',
]
# Columns to rename
rename_tree = {
    'tree_id': 'tree_id',
    'status': 'status',
    'health': 'health',
    'spc_common': 'species',
    'zipcode': 'zipcode',
    'zip_city': 'city',
    'latitude': 'latitude',
    'longitude': 'longitude',
    'x_sp': 'x_coord',
    'y_sp': 'y_coord',
}
# Column types
type_tree = {
    'tree_id': 'int',
    'status': 'str',
    'health': 'str',
    'species': 'str',
    'zipcode': 'int',
    'city': 'str',
    'latitude': 'float',
    'longitude': 'float',
    'x_coord': 'float',
    'y_coord': 'float',
}

df_tree_f = clean_filter(df_tree, use_tree, rename_tree, type_tree)

# Converting latitude and longitude into Geometry data
df_tree_f['geometry'] = df_tree_f.apply(lambda x: Point(float(x['longitude']), float(x['latitude'])), axis=1)

# Converting DataFrame to GeoDataFrame
gdf_tree_f = gpd.GeoDataFrame(df_tree_f, geometry='geometry')

# Setting the coordinate reference system
gdf_tree_f.set_crs(epsg=4326, inplace=True)
gdf_tree_f.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 11 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   tree_id    20 non-null     int64   
 1   status     20 non-null     object  
 2   health     20 non-null     object  
 3   species    20 non-null     object  
 4   zipcode    20 non-null     int64   
 5   city       20 non-null     object  
 6   latitude   20 non-null     float64 
 7   longitude  20 non-null     float64 
 8   x_coord    20 non-null     float64 
 9   y_coord    20 non-null     float64 
 10  geometry   20 non-null     geometry
dtypes: float64(4), geometry(1), int64(2), object(4)
memory usage: 1.8+ KB


In [12]:
# Zillow Rent Cleaning & Filtering
# Columns to remove
remove_rent = [
    'RegionID',
    'SizeRank',
    'RegionType',
    'StateName',
    'Metro',
    'CountyName',
]
# Columns to rename
rename_rent = {
    'RegionName': 'zipcode',
    'State': 'state',
    'City': 'city',
}
# Column types
type_rent = {
    'zipcode': 'int',
    'state': 'str',
    'city': 'str'
}

# Removing and renaming columns
df_rent_f = df_rent.drop(columns=remove_rent, axis=1)
df_rent_f = df_rent_f.rename(columns=rename_rent)
# Confirming Data type
for column, dtype in type_rent.items():
    df_rent_f[column] = df_rent_f[column].astype(dtype, errors='raise')
for column in df_rent_f.iloc[:, 3:]:
    df_rent_f[column] = df_rent_f[column].astype('float', errors='raise')
# Discarding blank data
df_rent_f.dropna(inplace=True)
df_rent_f.head()

,zipcode,state,city,2015-01-31,2015-02-28,2015-03-31,2015-04-30,2015-05-31,2015-06-30,2015-07-31,...,2022-12-31,2023-01-31,2023-02-28,2023-03-31,2023-04-30,2023-05-31,2023-06-30,2023-07-31,2023-08-31,2023-09-30
0,77494,TX,Katy,1606.206406,1612.779844,1622.201575,1630.392427,1632.411500,1636.206864,1644.894632,...,1994.653463,2027.438438,2042.237444,2049.325559,2016.531345,2023.438976,2031.558202,2046.144009,2053.486247,2055.771355
1,77449,TX,Katy,1257.814660,1255.268025,1262.170452,1274.955754,1285.526052,1295.665673,1296.650395,...,1749.697900,1738.217986,1747.305840,1758.407295,1758.891075,1762.980879,1771.751591,1779.338402,1795.384582,1799.631140
8,77433,TX,Cypress,1332.384333,1328.107408,1334.325290,1342.507107,1343.204774,1349.345048,1357.258039,...,1881.204550,1885.695935,1884.894986,1880.532012,1870.035369,1863.111029,1892.511066,1922.759295,1945.581823,1975.672556
15,11226,NY,New York,1944.609891,1971.608676,2044.189151,2061.734126,2076.492259,2057.344849,2075.751863,...,2695.164727,2680.683700,2676.791284,2697.414702,2706.080701,2726.381017,2753.966159,2770.403005,2785.320137,2762.435713
17,30044,GA,Lawrenceville,1157.569878,1168.554459,1173.937503,1176.423247,1180.259047,1183.618022,1193.861072,...,2125.698496,2113.964250,2170.556028,2160.008767,2191.894968,2181.780560,2161.711011,2117.990720,2119.319392,2055.476496


In [13]:
# Zipcode Cleaning & Filtering
# Columns to use
use_gdf = [
    'ZIPCODE',
    'PO_NAME',
    'STATE',
    'COUNTY',
    'geometry',
]
# Columns to rename
rename_gdf = {
    'ZIPCODE': 'zipcode',
    'PO_NAME': 'city',
    'STATE': 'state',
    'COUNTY': 'county',
    'geometry': 'geometry',
}
# Column types
type_gdf = {
    'zipcode': 'int',
    'city': 'str',
    'state': 'str',
    'county': 'str',
    'geometry': 'geometry',
}

gdf_zipcode_f = clean_filter(gdf_zipcode, use_gdf, rename_gdf, type_gdf)
gdf_zipcode_f.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 263 entries, 0 to 262
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   zipcode   263 non-null    int64   
 1   city      263 non-null    object  
 2   state     263 non-null    object  
 3   county    263 non-null    object  
 4   geometry  263 non-null    geometry
dtypes: geometry(1), int64(1), object(3)
memory usage: 10.4+ KB


## Part 2. Storing Data

In [14]:
def setup_new_postgis_database(user_name, db_name):
    """Dropping Database if already exists"""
    os.system(f"dropdb -U {user_name} --if-exists {db_name}")
    """Setting up a new PostGis Database"""
    os.system(f"createdb -U {user_name} {db_name}")
    os.system(f"psql -U {user_name} --dbname {db_name} -c 'CREATE EXTENSION postgis;'")

In [15]:
setup_new_postgis_database(db_username, db_name)

CREATE EXTENSION


### Creating Tables

In [16]:
engine = create_engine(db_url)

In [17]:
# Defining the SQL statements to create 311, tree, rent, and zipcode tables
nyc_311_schema = """
CREATE TABLE IF NOT EXISTS nyc_311 
(
    complaint_id INTEGER PRIMARY KEY,
    date DATE,
    complaint_type TEXT,
    zipcode INTEGER,
    city TEXT,
    x_coord INTEGER,
    y_coord INTEGER,
    latitude FLOAT,
    longitude FLOAT,
    geometry GEOMETRY(Point, 4326)
);

CREATE INDEX IF NOT EXISTS idx_nyc_311_geom ON nyc_311 USING GIST (geometry);
"""

nyc_tree_schema = """
CREATE TABLE IF NOT EXISTS nyc_tree
(
    tree_id INTEGER PRIMARY KEY,
    status TEXT,
    health TEXT,
    species TEXT,
    zipcode INTEGER,
    city TEXT,
    latitude FLOAT,
    longitude FLOAT,
    x_coord FLOAT,
    y_coord FLOAT,
    geometry GEOMETRY(Point, 4326)
);

CREATE INDEX IF NOT EXISTS idx_nyc_tree_geom ON nyc_tree USING GIST (geometry);
"""

nyc_rent_schema = """
CREATE TABLE IF NOT EXISTS nyc_rent
(
    zipcode INTEGER PRIMARY KEY,
    state TEXT,
    city TEXT
)
"""

nyc_zipcode_schema = """
CREATE TABLE IF NOT EXISTS nyc_zipcode
(
    zipcode INTEGER PRIMARY KEY,
    city TEXT,
    state TEXT,
    county TEXT,
    geometry GEOMETRY(Polygon, 4326)
);

CREATE INDEX IF NOT EXISTS idx_nyc_zipcode_geom ON nyc_zipcode USING GIST (geometry);
"""

In [18]:
# Executing the schema files to create tables
def create_table(engine, table_schema):
    """Creating a table using the SQL schema"""
    try:
        with engine.connect() as connection:
            connection.execute(text(table_schema))
        print("Table Created")
    except Exception as e:
        print(f"Error: {e}")

In [19]:
# Creating each table
create_table(engine, nyc_311_schema)
create_table(engine, nyc_tree_schema)
create_table(engine, nyc_rent_schema)
create_table(engine, nyc_zipcode_schema)

Table Created
Table Created
Table Created
Table Created


### Adding Data to Database

In [20]:
def write_dataframes_to_table(tablename_to_dataframe, engine):
    """Adding data to SQL table"""
    for table_name, df in tablename_to_dataframe.items():
        # Checking if the dataframe is a GeoDataframe
        if isinstance(df, gpd.GeoDataFrame):
            # Converting Shapely geometries to WKTElement
            df['geometry'] = df['geometry'].apply(
                lambda x: WKTElement(x.wkt, srid=4326) if isinstance(x, BaseGeometry) else x
            )
            geometry_type = 'GEOMETRY'  # Default type
            # GeoDataframe to SQL
            try:
                df.to_sql(table_name, con=engine, if_exists='replace', index=False,
                          dtype={'geometry': Geometry(geometry_type, srid=4326)})
                print(f"Data in {table_name}")
            except Exception as e:
                print(f"{table_name} Error: {e}")
        else:
            # Dataframe to SQL
            try:
                df.to_sql(table_name, con=engine, if_exists='replace', index=False)
                print(f"Data in {table_name}")
            except Exception as e:
                print(f"{table_name} Error: {e}")

In [21]:
gdf_zipcode_sql = gdf_zipcode_f.copy()
gdf_311_sql = gdf_311_f.copy()
gdf_tree_sql = gdf_tree_f.copy()
df_rent_sql = df_rent_f.copy()
# Dataframe to Table mapping
tablename_to_dataframe = {
    "nyc_zipcodes": gdf_zipcode_sql,
    "311_complaints": gdf_311_sql,
    "nyc_trees": gdf_tree_sql,
    "nyc_rents": df_rent_sql,
}

In [24]:
write_dataframes_to_table(tablename_to_dataframe, engine)

/Users/ariez/anaconda3/lib/python3.10/site-packages/geopandas/geodataframe.py:1537: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")
/Users/ariez/anaconda3/lib/python3.10/site-packages/geopandas/geodataframe.py:1537: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")
/Users/ariez/anaconda3/lib/python3.10/site-packages/geopandas/geodataframe.py:1537: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


Data in nyc_zipcodes
Data in 311_complaints
Data in nyc_trees
Data in nyc_rents


## Part 3: Understanding the Data

### Query 1

In [ ]:
# Helper function to write the queries to file
def write_query_to_file(query, outfile):
    raise NotImplementedError()

In [ ]:
QUERY_1_FILENAME = QUERY_DIR / "FILL_ME_IN"

QUERY_1 = """
FILL_ME_IN
"""

In [ ]:
with engine.connect() as conn:
    result = conn.execute(db.text(QUERY_1))
    for row in result:
        print(row)

In [ ]:
write_query_to_file(QUERY_1, QUERY_1_FILENAME)

## Part 4: Visualizing the Data

### Visualization 1

In [ ]:
# use a more descriptive name for your function
def plot_visual_1(dataframe):
    figure, axes = plt.subplots(figsize=(20, 10))
    
    values = "..."  # use the dataframe to pull out values needed to plot
    
    # you may want to use matplotlib to plot your visualizations;
    # there are also many other plot types (other 
    # than axes.plot) you can use
    axes.plot(values, "...")
    # there are other methods to use to label your axes, to style 
    # and set up axes labels, etc
    axes.set_title("Some Descriptive Title")
    
    plt.show()

In [ ]:
def get_data_for_visual_1():
    # Query your database for the data needed.
    # You can put the data queried into a pandas/geopandas dataframe, if you wish
    raise NotImplementedError()

In [ ]:
some_dataframe = get_data_for_visual_1()
plot_visual_1(some_dataframe)